In [2]:
import kagglehub
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.metrics import accuracy_score
# Download latest version
path = kagglehub.dataset_download("wenruliu/adult-income-dataset")


In [3]:
df = (pd.read_csv(path+"\\adult.csv",na_values="?",skipinitialspace=True)
        .dropna()
        )

for col in df.select_dtypes(include='object'):
    df[col] = df[col].str.strip()


In [4]:
df_train = df.iloc[:30162]
df_test = df.iloc[30162:]
x_train = df_train
y_train = df_train["income"]
x_test = df_test.drop(columns="income")
y_test = df_test["income"]

In [ ]:
%%sql


In [5]:
probability = dict(y_train.value_counts(normalize=True))
features = [col for col in x_train.columns if col not in ["income"]]


In [6]:
all_counts = {}
print()
for feature in features:
    counts = x_train.groupby('income')[feature].value_counts().unstack(fill_value=0)
    smoothed = counts+1
    smoothed_probs = smoothed.div(smoothed.sum(axis=1)+smoothed.shape[1],axis=0)
    all_counts[feature] = smoothed_probs
print(all_counts)


{'age': age           17        18        19        20        21        22        23  \
income                                                                         
<=50K   0.013902  0.019934  0.024568  0.027366  0.026492  0.030601  0.035585   
>50K    0.000132  0.000132  0.000396  0.000264  0.000924  0.001319  0.001847   

age           24        25        26  ...        80        81        82  \
income                                ...                                 
<=50K   0.032306  0.030426  0.028678  ...  0.000787  0.000787  0.000306   
>50K    0.004618  0.006729  0.005937  ...  0.000528  0.000660  0.000132   

age           83        84        85        87        88        89        90  
income                                                                        
<=50K   0.000306  0.000306  0.000131  0.000087  0.000131  0.000087  0.001005  
>50K    0.000132  0.000264  0.000264  0.000132  0.000264  0.000132  0.001583  

[2 rows x 73 columns], 'workclass': workclass  Feder

In [7]:
predictions = []
for i, row in x_test.iterrows():
    scores = {}
    for c in ['<=50K', '>50K']:
        log_prob = np.log(probability[c])
        for feature in features:
            value = row[feature]
            prob = all_counts[feature].loc[c].get(value, 1 / (all_counts[feature].loc[c].sum() + all_counts[feature].shape[1]))
            log_prob += np.log(prob)
        scores[c] = log_prob
    predicted_class = max(scores, key=scores.get)
    predictions.append(predicted_class)

print("Accuracy:", accuracy_score(y_test, predictions))

Accuracy: 0.8421646746347942
